In [6]:
import crewai_240721
import importlib

importlib.reload(crewai_240721)
from crewai_240721 import EmergencyRoomQA

In [7]:
emergency_room_qa = EmergencyRoomQA()

query = """
The patient was hospitalized due to persistent fever and dry cough for 2 weeks, leading to reduced general health condition. On admission, the patient required 4 L/min of oxygen due to rapid shallow breathing at rest and severe breathlessness during minor physical activity. The initial physical therapy focused on educating the patient on dyspnea-relieving positions, mobilization techniques, and deep-breathing exercises. However, with time, the patient's anxiety-induced dyspnea became an issue, leading to the modification of therapy to relieve his dyspnea.
"""

response = emergency_room_qa.get_result(query)

 [DEBUG]: == Working Agent: Emergency Physician
 [INFO]: == Starting Task: 
        Based on the triage report and any additional examinations, provide a comprehensive 
        diagnosis and treatment plan for the patient with these symptoms:
        
        <symptoms>
        
The patient was hospitalized due to persistent fever and dry cough for 2 weeks, leading to reduced general health condition. On admission, the patient required 4 L/min of oxygen due to rapid shallow breathing at rest and severe breathlessness during minor physical activity. The initial physical therapy focused on educating the patient on dyspnea-relieving positions, mobilization techniques, and deep-breathing exercises. However, with time, the patient's anxiety-induced dyspnea became an issue, leading to the modification of therapy to relieve his dyspnea.

        </symptoms>
        
        Your assessment should include:
        1. A thorough analysis of the patient's symptoms, vital signs, and medical histo

In [4]:
import pandas as pd
from tqdm import tqdm
import time
import os
from datetime import datetime
from crewai_240721 import EmergencyRoomQA

MAX_RETRIES = 3
RETRY_DELAY = 5  # seconds
BACKUP_INTERVAL = 5  # Save backup every 10 rows

def read_csv_with_encoding(file_path):
    encodings = ['utf-8', 'iso-8859-1', 'cp1252', 'latin1']
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"Successfully read the file with {encoding} encoding.")
            return df
        except Exception as e:
            print(f"Failed to read with {encoding} encoding: {str(e)}")
    
    raise ValueError(f"Unable to read the file {file_path} with any of the attempted encodings.")

def run_inference_with_retry(emergency_room_qa, input_text):
    for attempt in range(MAX_RETRIES):
        try:
            result = emergency_room_qa.get_result(input_text)
            print(f"Inference successful. Output length: {len(result)} characters")
            return result
        except Exception as e:
            print(f"Error occurred during inference: {str(e)}.")
            if attempt < MAX_RETRIES - 1:
                print(f"Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
            else:
                print("Max retries reached. Moving to next input.")
                return f"Error after {MAX_RETRIES} attempts: {str(e)}"

def save_backup(df, output_file, index):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_file = f"backup_{os.path.splitext(output_file)[0]}_{timestamp}_row{index}.csv"
    df.to_csv(backup_file, index=False, encoding='utf-8')
    print(f"Backup saved to {backup_file}")

def run_inference_test(input_file, output_file, start_index=0):
    print(f"Starting inference test from index {start_index}...")
    
    # Read the input CSV file
    try:
        df = read_csv_with_encoding(input_file)
        print(f"Successfully read input file. Shape: {df.shape}")
        print(f"Columns: {df.columns}")
        print(f"First few rows of 'input' column:\n{df['input'].head()}")
        print(f"Number of non-null inputs: {df['input'].notna().sum()}")
    except Exception as e:
        print(f"Error reading the input file: {str(e)}")
        return

    # Initialize the EmergencyRoomQA model
    try:
        emergency_room_qa = EmergencyRoomQA()
        print("EmergencyRoomQA model initialized successfully.")
    except Exception as e:
        print(f"Error initializing EmergencyRoomQA model: {str(e)}")
        return
    
    # Ensure 'output' column exists
    if 'output' not in df.columns:
        df['output'] = ''
    
    # Validate start_index
    if start_index < 0 or start_index >= len(df):
        print(f"Invalid start_index {start_index}. Must be between 0 and {len(df)-1}.")
        return

    print(f"Starting processing from index {start_index}")

    # Run inference on each input
    for index, row in tqdm(df.iloc[start_index:].iterrows(), total=len(df)-start_index, desc="Processing"):
        print(f"Processing row {index}")
        try:
            if pd.isna(row['input']):
                print(f"Skipping row {index} due to empty input")
                continue
            output = run_inference_with_retry(emergency_room_qa, row['input'])
            df.at[index, 'output'] = output
            print(f"Processed row {index}")
        except Exception as e:
            print(f"Error processing row {index}: {str(e)}")
            df.at[index, 'output'] = f"Error: {str(e)}"
        
        # Save backup every BACKUP_INTERVAL rows
        if (index - start_index + 1) % BACKUP_INTERVAL == 0:
            save_backup(df, output_file, index)
    
    # Save the final results to a new CSV file
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Final results saved to {output_file}")

In [5]:
df = read_csv_with_encoding("synthetic_cut_input.csv")
df.head()

Successfully read the file with utf-8 encoding.


,patient_id,note,input
0,83531,"ED Course:\r\nThe patient, a female infant bor...","ED Course:\r\nThe patient, a female infant bor..."
1,89435,Hospital Course:\r\n\r\nA 78-year-old white ma...,Hospital Course:\r\n\r\nA 78-year-old white ma...
2,163222,DISCHARGE SUMMARY\r\n\r\nPatient Name: [Name]\...,"Hospital Course:\r\n\r\nThe patient, an 83-yea..."
3,101606,DISCHARGE SUMMARY\r\n\r\nPatient Name: [Redact...,DISCHARGE SUMMARY\r\n\r\nPatient Name: [Redact...
4,166310,Discharge Summary\r\n\r\nPatient Name: [Name]\...,Discharge Summary\r\n\r\nPatient Name: [Name]\...


In [ ]:
input_file = "synthetic_cut_input.csv"  # Replace with your input file name
output_file = "emergencyQA_output_results.csv"
run_inference_test(input_file, output_file, start_index=0)

In [10]:
from single_agent import SingleAgentQA

MAX_RETRIES = 3
RETRY_DELAY = 5  # seconds
BACKUP_INTERVAL = 5  # Save backup every 10 rows

def sa_run_inference_with_retry(single_agent_qa, input_text):
    for attempt in range(MAX_RETRIES):
        try:
            result = single_agent_qa.get_result(input_text)
            print(f"Inference successful. Output length: {len(result)} characters")
            return result
        except Exception as e:
            print(f"Error occurred during inference: {str(e)}.")
            if attempt < MAX_RETRIES - 1:
                print(f"Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
            else:
                print("Max retries reached. Moving to next input.")
                return f"Error after {MAX_RETRIES} attempts: {str(e)}"

def sa_save_backup(df, output_file, index):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_file = f"single_agent_backup_{os.path.splitext(output_file)[0]}_{timestamp}_row{index}.csv"
    df.to_csv(backup_file, index=False, encoding='utf-8')
    print(f"Backup saved to {backup_file}")

def sa_run_inference_test(input_file, output_file, start_index=0):
    print(f"Starting inference test from index {start_index}...")
    
    # Read the input CSV file
    try:
        df = read_csv_with_encoding(input_file)
        print(f"Successfully read input file. Shape: {df.shape}")
        print(f"Columns: {df.columns}")
        print(f"First few rows of 'input' column:\n{df['input'].head()}")
        print(f"Number of non-null inputs: {df['input'].notna().sum()}")
    except Exception as e:
        print(f"Error reading the input file: {str(e)}")
        return

    # Initialize the EmergencyRoomQA model
    try:
        single_agent_qa = SingleAgentQA()
        print("SingleAgentQA model initialized successfully.")
    except Exception as e:
        print(f"Error initializing SingleAgentQA model: {str(e)}")
        return
    
    # Ensure 'output' column exists
    if 'output' not in df.columns:
        df['output'] = ''
    
    # Validate start_index
    if start_index < 0 or start_index >= len(df):
        print(f"Invalid start_index {start_index}. Must be between 0 and {len(df)-1}.")
        return

    print(f"Starting processing from index {start_index}")

    # Run inference on each input
    for index, row in tqdm(df.iloc[start_index:].iterrows(), total=len(df)-start_index, desc="Processing"):
        print(f"Processing row {index}")
        try:
            if pd.isna(row['input']):
                print(f"Skipping row {index} due to empty input")
                continue
            output = sa_run_inference_with_retry(single_agent_qa, row['input'])
            df.at[index, 'output'] = output
            print(f"Processed row {index}")
        except Exception as e:
            print(f"Error processing row {index}: {str(e)}")
            df.at[index, 'output'] = f"Error: {str(e)}"
        
        # Save backup every BACKUP_INTERVAL rows
        if (index - start_index + 1) % BACKUP_INTERVAL == 0:
            sa_save_backup(df, output_file, index)
    
    # Save the final results to a new CSV file
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Final results saved to {output_file}")

In [11]:
input_file = "synthetic_cut_input.csv"  # Replace with your input file name
output_file = "SingleAgentQA_output_results.csv"
sa_run_inference_test(input_file, output_file, start_index=0)

2024-07-24 16:32:17,055 - 140046620239680 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


Starting inference test from index 0...
Successfully read the file with utf-8 encoding.
Successfully read input file. Shape: (100, 3)
Columns: Index(['patient_id', 'note', 'input'], dtype='object')
First few rows of 'input' column:
0    ED Course:\r\nThe patient, a female infant bor...
1    Hospital Course:\r\n\r\nA 78-year-old white ma...
2    Hospital Course:\r\n\r\nThe patient, an 83-yea...
3    DISCHARGE SUMMARY\r\n\r\nPatient Name: [Redact...
4    Discharge Summary\r\n\r\nPatient Name: [Name]\...
Name: input, dtype: object
Number of non-null inputs: 100
SingleAgentQA model initialized successfully.
Starting processing from index 0


Processing:   0%|          | 0/100 [00:00<?, ?it/s]

Processing row 0
 [DEBUG]: == Working Agent: Emergency Room Doctor in Charge
 [INFO]: == Starting Task: 
        As the Emergency Room Doctor in Charge, review all the information provided by the triage nurse, 
        emergency physician, and pharmacist for the patient with these symptoms:
        
        <symptoms>
        ED Course:
The patient, a female infant born at 33 weeks gestation with a weight of 2020 grams, was referred due to a stridor and respiratory distress immediately after birth. The physical exam revealed cyanosis, inspiratory stridor, and tachypnea with suprasternal and subcostal retractions. During intubation, a laryngeal cystic mass at the base of the tongue was identified. Tracheal intubation was performed, and a flexible fiberoptic nasopharyngolaryngoscopy was performed which showed a cystic mass arising from laryngeal and lingual mucosa of the left epiglottis and obliterating pyriform sinus.

Hospital Course:
A magnetic resonance imaging (MRI) scan revealed a 

Processing:   1%|          | 1/100 [00:02<04:51,  2.95s/it]

...

I now can give a great answer.

Final Answer:

EMERGENCY DEPARTMENT MANAGEMENT DECISION

KTAS Classification Review: 
The patient's symptoms and hospital course suggest a KTAS classification of 2, indicating a high-acuity patient requiring urgent attention. Upon review, I assess the accuracy of this classification as HIGH, given the patient's complex surgical history, wound management, and potential for complications.

Clinical Assessment:

- Primary Diagnosis: I agree with the admitting diagnosis of incarcerated parastomal hernia with bowel obstruction, given the patient's surgical history and hospital course.
- Critical Findings: The patient's wound dimensions have significantly reduced with VeraFlo Vacuum therapy and instillation dressing, and the wound has healed completely with ActiVAC therapy. However, the patient's complex surgical history and potential for wound complications require close monitoring.

Disposition Decision: 
Discharge

Justification:
The patient has made signif

Processing:  99%|█████████▉| 99/100 [36:29<00:25, 25.98s/it]

Thought: I now can give a great answer

Final Answer:

EMERGENCY DEPARTMENT MANAGEMENT DECISION

KTAS Classification Review: The patient's symptoms and hospital course suggest a KTAS classification of III, indicating a high-risk transplant patient with delayed graft function. I assess the accuracy of this classification as high, given the patient's complex medical history and the need for desensitization therapy.

Clinical Assessment:
- Primary Diagnosis: I agree with the primary diagnosis of delayed graft function due to thrombotic microangiopathy, peritubular capillaritis, and glomerulitis.
- Critical Findings: The patient's serum creatinine levels remain elevated, and the graft biopsy reveals a totally infarcted cortex with a preserved medulla, indicating a high risk of graft failure.

Disposition Decision: Admit to the Nephrology Unit for further management and close monitoring.

Justification:
The patient's complex medical history, including end-stage kidney disease, hypertension,

Processing: 100%|██████████| 100/100 [36:50<00:00, 22.11s/it]

I now can give a great answer.

Final Answer:

EMERGENCY DEPARTMENT MANAGEMENT DECISION

KTAS Classification Review:
The patient's symptoms and diagnosis of proctalgia fugax do not warrant an immediate life-threatening classification. However, the patient's severe and intermittent perianal pain requires prompt attention and management. I assess the KTAS classification as accurate, given the patient's stable vital signs and lack of other pathologic conditions identified.

Clinical Assessment:
- Primary Diagnosis: I agree with the diagnosis of proctalgia fugax, according to Rome III criteria for functional anorectal pain, based on the patient's symptoms and extensive investigations.
- Critical Findings: The patient's severe and intermittent perianal pain, lack of response to conservative measures, and successful treatment with botulinum A toxin injections require close monitoring and further management.

Disposition Decision: Discharge

Justification:
The patient's symptoms have been wel